# Deutsch-Jozsa Algorithm #

In [ ]:
# imports
import os;
import sys;

# NOTE: need this to force jupyter to reload imports:
for key in list(sys.modules.keys()):
    if key.startswith('src.'):
        del sys.modules[key];

os.chdir(os.path.dirname(_dh[0]));
sys.path.insert(0, os.getcwd());

from src.thirdparty.maths import *;
from src.thirdparty.misc import *;
from src.thirdparty.quantum import *;
from src.thirdparty.render import *;

from src.widgets import *;
from src.api.ibm import *;
from src.api.statistics import *;
from src.algorithms import *;

np.random.seed(39102901); # for repeatability

In [ ]:
n = 4;
oraclenr = np.random.randint(0, 2);
circuit = deutsch_jozsa_algorithm(n=n, oraclenr=oraclenr, verbose=True);
display(circuit.draw(output=DRAW_MODE.COLOUR.value, cregbundle=False, initial_state=True));

In [ ]:
@connect_to_backend(kind=BACKEND_SIMULATOR.AER)
def action(kind, backend):
    shots = 10000;
    circuit = deutsch_jozsa_algorithm(n=n, verbose=True);
    job = qk_execute(experiments=circuit, backend=backend, shots=shots);
    result = job.result();
    counts, _ = get_counts(result);
    display(QkVisualisation.plot_histogram(counts));
action();

In [ ]:
# Run with cloud computing
NUM_SHOTS = 20000;
@connect_to_backend(kind=BACKEND.LEAST_BUSY, n=n+1)
def action(kind, backend):
    circuit = deutsch_jozsa_algorithm(n=n);
    %qiskit_job_watcher
    job = qk_execute(experiments=circuit, backend=backend, shots=NUM_SHOTS, optimization_level=3);
    print(dedent(f'''
        \x1b[1mNOTE:\x1b[0m
        - 'least busy' backend: \x1b[1m{str(backend)}\x1b[0m
        - job index:            \x1b[1m{job.job_id()}\x1b[0m
    '''));
action();

In [ ]:
# extract statitics:
@recover_job(ensure_job_done=True)
def action(job: IBMQJob):
    result = job.result();
    counts, _ = get_counts(result);
    display(
        QkVisualisation.plot_histogram(counts, title='Measurements'),
    );

action();